## dCMF
Example of running the "dcmf" module with the use provided parameters

In [1]:
import sys
sys.path.append("..")

In [2]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os
from datetime import datetime

In [3]:
from src.dcmf import dcmf

## Loading the sample dataset

This directory contains a sample synthetic dataset generated for the augmented setting of Fig 1(c) in the [paper](https://arxiv.org/abs/1811.11427).
You can download the sample data from [here](https://drive.google.com/open?id=1EFF_kuOIg2aYyOGZY_peX3NziqCSxxP1) and unzip it to the data directory.

In [4]:
sample_no = 3
data_dir = f"../data/CellLine/sample{sample_no}/"

In [5]:
#Loads the dataset into a dict
#Note: This dataset contains 5-folds for the matrix X_12 (matrix R below)
num_folds = 1
#
pp = pprint.PrettyPrinter()
print("Loading data from data_dir: ",data_dir)
U1 = pkl.load(open(data_dir+"X_12_mut.pkl",'rb'))
V1 = pkl.load(open(data_dir+"X_12_cnv.pkl",'rb'))
# V1 = pkl.load(open(data_dir+"X_26.pkl",'rb'))
W1 = pkl.load(open(data_dir+"X_12_gene_exp.pkl",'rb'))
R_temp_dict = {}
for fold_num in np.arange(1,num_folds+1):
    Rtrain = pkl.load(open(data_dir+'/X_13_train_fold_'+str(fold_num)+'.pkl','rb'))
    Rtrain = Rtrain
    Rtrain_idx = pkl.load(open(data_dir+'/X_13_train_idx_'+str(fold_num)+'.pkl','rb')) 
    Rtest = pkl.load(open(data_dir+'/X_13_test_fold_'+str(fold_num)+'.pkl','rb'))
    Rtest_idx = pkl.load(open(data_dir+'/X_13_test_idx_'+str(fold_num)+'.pkl','rb'))
    Rdoublets = pkl.load(open(data_dir+'/R_doublets_'+str(fold_num)+'.pkl','rb'))
    R_temp_dict[fold_num] = {"Rtrain":Rtrain,"Rtrain_idx":Rtrain_idx,"Rtest":Rtest,"Rtest_idx":Rtest_idx,"Rdoublets":Rdoublets}
#
data_dict = {"U1":U1,"V1":V1, "W1":W1, "R":R_temp_dict}

Loading data from data_dir:  ../data/CellLine/sample3/


In [6]:
print("U1.shape: ",U1.shape)
print("V1.shape: ",V1.shape)
print("W1.shape: ",W1.shape)
print("R.shape: ",data_dict['R'][1]['Rtrain'].shape)

U1.shape:  (692, 638)
V1.shape:  (692, 638)
W1.shape:  (692, 638)
R.shape:  (692, 396)


## Building the required data structures

Here we construct the data structures required as input to the dcmf API

#### *entity matrix relationship graph *

- **G**: dict, keys are entity IDs and values are lists of associated matrix IDs

#### * training data*
- **X_data**: dict, keys are matrix IDs and values are (1) np.array, or (2) dict, (if this matrix is in validation set **X_val**) with validation set IDs as keys & values as np.array
- **X_meta**: dict, keys are matrix IDs and values are lists of the 2 associated entity IDs

#### *validation data*
- **X_val**: dict, keys are IDs of the matrices that are part of validation set and values are dict with validation set IDs as keys and values are (1) scipy.sparse matrix, or (2) list of triplets corresponding to the validation entries (if you would like to perform classification and measure AUC)  
**Note**: To perform K folds cross validation, use K validation sets for the corresponsing matrix/matrices. In the example below, we used a single validation set with ID "1" for each of the matrices with IDs "X1" and "X2"

In [7]:
G = {
    "e1":["X1","X2","X3","X4"],\
    "e2":["X1","X2","X3"],\
    "e3":["X4"]
}

In [8]:
X_data = {
    "X1":{"1":U1},\
    "X2":{"1":V1},\
    "X3":{"1":W1},\
    "X4":{"1":data_dict['R'][1]["Rtrain"]},\
    }

In [9]:
X_meta = {
    "X1":["e1","e2"],\
    "X2":["e1","e2"],\
    "X3":["e1","e2"],\
    "X4":["e1",'e3']}
    #"X5":["e5","e3"]}

In [10]:
# Rtest_triplets1 = [[1,1,1],[2,2,0]]
# Rtest_triplets2 = [[1,1,1],[3,3,0],[1,2,0],[0,1,0],[0,2,0],[0,3,0]]
Rtest_triplets1 = [[0, 30, 1], [0, 20, 0]]
Rtest_triplets2 = [[0, 0, 1], [0, 3, 0]]
Rtest_triplets3 = [[0, 0, 1], [0, 10, 0]]
Rtest_triplets4 = [[0, 1, 1], [0, 0, 0]]

In [11]:
X_val = {
    "X1":{"1":Rtest_triplets1},
    "X2":{"1":Rtest_triplets2},
    "X3":{"1":Rtest_triplets3},
    "X4":{"1":Rtest_triplets4}
}

#### *dCMF network construction - hyperparameters*

- **kf**: float, in the range (0,1) 
- **k**: int, entity representation or encoding size. Refer Appendix A in the [paper](https://arxiv.org/abs/1811.11427) for info about how k and kf are used in the dCMF network construction. 
- **e_actf**: str, autoencoder's encoding activation function.
- **d_actf**: str, autoencoder's decoding activation function. Supported functions are "tanh","sigma","relu","lrelu"
- **is_linear_last_enc_layer**: bool, True to set linear activation for the bottleneck/encoding generation layer 
- **is_linear_last_dec_layer**: bool, True to set linear activation for the output/decoding generation layer 
- **num_chunks**: int, number of training batches to create.

In [12]:
kf = 0.5
k = 100
e_actf = "tanh"
d_actf = "tanh"
is_linear_last_enc_layer = False
is_linear_last_dec_layer = False
num_chunks = 2

#### *Optimization/training - hyperparamteres*

- **learning_rate**: float, Adam optimizer's learning rate
- **weight_decay**: float, Adam optimizers's weight decay (L2 penalty)
- **max_epochs**: int, maximum number of training epochs at which the training stops 
- **convg_thres**: float, convergence threshold 

In [13]:
learning_rate = 0.001
weight_decay = 0.05
max_epochs = 5
convg_thres = 0.1

#### *Hyperparamteres related to pre-training*

- **is_pretrain**: bool, True for pretraining 
- **pretrain_thres**: bool, pre-training convergence thresholsd
- **max_pretrain_epochs**: int, maximum number of pre-training epochs at which the training stops

In [14]:
is_pretrain=True
pretrain_thres= 0.1
max_pretrain_epochs = 2

#### *Parameters related to validation*

- **val_metric**: str, Validation performance metric. Supported metrics: ["rmse","r@k","p@k","auc"]. Where,  
     *rmse* - Root [mean square error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)  
     *r@k* - Recall@k. Refer section 5.2's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *p@k* - Probability@k. Refer section 5.3's sub-section "Evaluation metric" in the [paper](https://arxiv.org/abs/1811.11427)      
     *auc* - [Area under the curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    
- **is_val_transpose**: bool, True if the reconstructed matrix has to be transposed before computing the validation performance
- **at_k**: int, the value of k if the **val_metric** is either "r@k" or "p@k"

In [15]:
val_metric = "auc"
is_val_transpose = True
at_k = 10

#### *GPU - parameters *

- **is_gpu**: bool, True if pytorch tensors storage and operations has to be done in GPU
- **gpu_ids**: str, Comma separated string of CUDA GPU ID

In [16]:
is_gpu = False
gpu_ids = "1"

#### *Instantiating the dCMF model...*
- Initializes dCMF after validating the input data and the (hyper)parameters

In [17]:
dcmf_model = dcmf(G, X_data, X_meta,\
            num_chunks=num_chunks,k=k, kf=kf, e_actf=e_actf, d_actf=d_actf,\
            learning_rate=learning_rate, weight_decay=weight_decay, convg_thres=convg_thres, max_epochs=max_epochs,\
            is_gpu=is_gpu,gpu_ids=gpu_ids,is_pretrain=is_pretrain, pretrain_thres=pretrain_thres,\
            max_pretrain_epochs=max_pretrain_epochs,X_val=X_val,val_metric=val_metric,\
            is_val_transpose=is_val_transpose, at_k=at_k,\
            is_linear_last_enc_layer=is_linear_last_enc_layer,is_linear_last_dec_layer=is_linear_last_dec_layer,num_val_sets=num_folds)

dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF:
---
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  3
num matrices:  4
num_val_sets:  1
X_val #matrices:  4
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#


#### *Fitting... *
- Performs the input transformation and network construction
- (Pre-trains and) trains the model to obtain the entity representations
- Reconstruct the input matrices using the entity representations obtained

In [18]:
start_time = datetime.now()

In [19]:
dcmf_model.fit()

## fold_num:  1  ##
dcmf_base.__init__ - start
dcmf_base.__init__ - end
#
dCMF: 
#
learning_rate:  0.001
weight_decay:  0.05
convg_thres:  0.1
max_epochs:  5
isPretrain:  True
pretrain_thres:  0.1
max_pretrain_epochs:  2
num_chunks:  2
k:  100
kf:  0.5
e_actf:  tanh
d_actf:  tanh
is_gpu:  False
gpu_ids:  1
num entities:  3
num matrices:  4
num_val_sets:  1
X_val #matrices:  4
val_metric (used only if X_val #matrices > 0):  auc
at_k (used only if X_val #matrices > 0 and val_metric is r@k or p@k):  10
is_val_transpose:  True
is_linear_last_enc_layer:  False
is_linear_last_dec_layer:  False
#
dcmf - model construction - start
__input_transformation - start
#
concatenated-matrix construction...
e_id:  e1
X_id_list:  ['X1', 'X2', 'X3', 'X4']
X_id:  X1
X[X_id].shape:  (692, 638)
X_id:  X2
X[X_id].shape:  (692, 638)
X_id:  X3
X[X_id].shape:  (692, 638)
X_id:  X4
X[X_id].shape:  (692, 396)
C_dict[e].shape:  torch.Size([692, 2310])
---
e_id:  e2
X_id_list:  ['X1', 'X2', 'X3']
X_id:  X1
X[X_id].

In [20]:
end_time = datetime.now()
runtime = end_time - start_time
runtime_seconds = runtime.total_seconds()
print(f"Total runtime = {runtime_seconds}")

Total runtime = 7.043905


#### *Result attributes:*
- **out_dict_U**:  dict, keys are validation set IDs and values are dict with entity IDs as keys and np.array of entity representations/encodings as values
- **out_dict_X_prime**: dict, keys are matrix IDs and values are matrix reconstructions
- **out_dict_info**: dict, keys are loss/validation performance attributes and values are corresponding results.

In [21]:
dcmf_model.out_dict_U['1'].keys()

dict_keys(['e1', 'e2', 'e3'])

In [22]:
dcmf_model.out_dict_X_prime['1'].keys()

dict_keys(['X1', 'X2', 'X3', 'X4'])

In [23]:
dcmf_model.out_dict_info

{'params': {'learning_rate': 0.001,
  'weight_decay': 0.05,
  'convg_thres': 0.1,
  'max_epochs': 5,
  'is_pretrain': True,
  'pretrain_thres': 0.1,
  'max_pretrain_epochs': 2,
  'num_chunks': 2,
  'k': 100,
  'kf': 0.5,
  'e_actf': 'tanh',
  'd_actf': 'tanh',
  'is_linear_last_enc_layer': False,
  'is_linear_last_dec_layer': False},
 'num_val_sets': 1,
 'loss_all_folds': {'1': [10.07929515838623,
   12.335293292999268,
   0.08367757499217987,
   5.284323751926422,
   6.113111734390259,
   19.525826930999756,
   0.07679611071944237]},
 'loss_all_folds_avg_tuple': [10.07929515838623,
  12.335293292999268,
  0.08367757499217987,
  5.284323751926422,
  6.113111734390259,
  19.525826930999756,
  0.07679611071944237],
 'loss_all_folds_avg_sum': 53.49832455441356,
 'val_metric': 'auc',
 'val_perf_all_folds': {'1': {'X1': 1.0, 'X2': 0.0, 'X3': 0.0, 'X4': 0.0}},
 'val_perf_all_folds_avg': {'X1': 1.0, 'X2': 0.0, 'X3': 0.0, 'X4': 0.0},
 'val_perf_all_folds_total': {'1': 1.0},
 'val_perf_all_fold

In [24]:
cell_line_emb = list(dcmf_model.out_dict_U['1'].values())[0].detach().numpy()
gene_emb = list(dcmf_model.out_dict_U['1'].values())[1].detach().numpy()
drug_emb = list(dcmf_model.out_dict_U['1'].values())[2].detach().numpy()

In [25]:
import pandas as pd

In [26]:
cl_emb_df = pd.DataFrame(cell_line_emb)
cl_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.073603,-0.040147,0.163516,0.281062,-0.040905,0.038282,-0.043648,0.109117,-0.399257,0.204860,...,-0.071574,-0.526656,0.070362,0.117407,-0.487201,0.053442,-0.172819,-0.160130,0.017123,-0.156740
1,-0.073481,-0.041415,0.163373,0.280411,-0.039475,0.039434,-0.042652,0.108264,-0.398858,0.205306,...,-0.070375,-0.525774,0.069531,0.117914,-0.487648,0.052837,-0.173062,-0.160275,0.018927,-0.157607
2,-0.074092,-0.040984,0.162482,0.280324,-0.039301,0.040021,-0.042367,0.108368,-0.399625,0.205083,...,-0.070573,-0.525542,0.069686,0.117948,-0.487376,0.052421,-0.173298,-0.159680,0.018849,-0.156957
3,-0.074232,-0.040839,0.163743,0.280667,-0.039474,0.039100,-0.042722,0.109069,-0.399573,0.204461,...,-0.070994,-0.526048,0.069511,0.118016,-0.487334,0.052818,-0.172470,-0.160205,0.018792,-0.157210
4,-0.072722,-0.040614,0.162553,0.280708,-0.041595,0.039035,-0.042355,0.108250,-0.399030,0.205839,...,-0.070587,-0.526335,0.069912,0.117160,-0.487788,0.053419,-0.173038,-0.159827,0.017888,-0.156648


In [27]:
cl_emb_df.shape

(692, 100)

In [28]:
gene_emb_df = pd.DataFrame(gene_emb)
gene_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.051363,-0.192645,-0.166976,0.141274,0.164899,-0.227068,-0.083322,-0.414564,0.185399,0.128922,...,0.188523,-0.162107,-0.364474,0.245074,0.452578,-0.227461,0.067002,0.048270,-0.072067,-0.045851
1,-0.049325,-0.191142,-0.170379,0.137041,0.165199,-0.231807,-0.082041,-0.412448,0.181589,0.128726,...,0.193388,-0.162707,-0.364038,0.242651,0.452966,-0.228371,0.068512,0.049113,-0.071517,-0.042087
2,-0.051381,-0.194705,-0.165266,0.143490,0.162897,-0.224201,-0.083369,-0.413871,0.187697,0.129706,...,0.188610,-0.163848,-0.365533,0.247314,0.452423,-0.225733,0.068108,0.046690,-0.070840,-0.047719
3,-0.050060,-0.192214,-0.171594,0.135916,0.165965,-0.232318,-0.081769,-0.413056,0.182242,0.129673,...,0.190673,-0.160819,-0.363417,0.239877,0.453075,-0.228421,0.067229,0.049147,-0.069625,-0.041169
4,-0.050288,-0.191256,-0.168559,0.140238,0.165215,-0.230080,-0.082241,-0.413237,0.184690,0.130632,...,0.189895,-0.161534,-0.363947,0.243697,0.452592,-0.227848,0.067165,0.047225,-0.070567,-0.043431


In [29]:
gene_emb_df.shape

(638, 100)

In [30]:
drug_emb_df = pd.DataFrame(drug_emb)
drug_emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.046993,0.063267,-0.124835,0.090900,-0.172522,0.053719,0.179705,-0.073574,-0.152063,0.041131,...,0.041171,-0.113993,0.050652,-0.038211,0.052388,-0.083036,0.107707,-0.021954,-0.308908,0.095162
1,-0.045586,-0.025346,-0.023065,0.009522,-0.086102,0.013557,0.081376,0.000746,-0.081976,0.032305,...,0.029315,-0.079234,0.004578,0.030469,0.054361,-0.029263,0.018625,-0.066893,-0.156494,0.047467
2,-0.030353,0.009752,-0.045525,0.034471,-0.120372,0.014057,0.120300,-0.005938,-0.109538,0.057558,...,0.019963,-0.075361,0.031781,0.005043,0.019450,-0.066644,0.044030,-0.039589,-0.202986,0.079133
3,-0.022353,0.034922,-0.117882,0.055130,-0.155105,0.023912,0.162207,-0.061717,-0.111846,0.064957,...,0.038977,-0.133693,0.029810,-0.021669,0.018235,-0.097860,0.052657,-0.023249,-0.276092,0.087677
4,-0.032243,-0.009500,-0.053250,0.048576,-0.133756,0.035284,0.111777,-0.017838,-0.112535,0.036076,...,0.046327,-0.109995,0.022048,0.020719,0.034716,-0.054137,0.046440,-0.010490,-0.207259,0.081143


In [31]:
drug_emb_df.shape

(396, 100)

In [32]:
emb_df = pd.concat([cl_emb_df, gene_emb_df, drug_emb_df], ignore_index = True, axis = 0)
emb_df.shape

(1726, 100)

In [33]:
emb_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.073603,-0.040147,0.163516,0.281062,-0.040905,0.038282,-0.043648,0.109117,-0.399257,0.204860,...,-0.071574,-0.526656,0.070362,0.117407,-0.487201,0.053442,-0.172819,-0.160130,0.017123,-0.156740
1,-0.073481,-0.041415,0.163373,0.280411,-0.039475,0.039434,-0.042652,0.108264,-0.398858,0.205306,...,-0.070375,-0.525774,0.069531,0.117914,-0.487648,0.052837,-0.173062,-0.160275,0.018927,-0.157607
2,-0.074092,-0.040984,0.162482,0.280324,-0.039301,0.040021,-0.042367,0.108368,-0.399625,0.205083,...,-0.070573,-0.525542,0.069686,0.117948,-0.487376,0.052421,-0.173298,-0.159680,0.018849,-0.156957
3,-0.074232,-0.040839,0.163743,0.280667,-0.039474,0.039100,-0.042722,0.109069,-0.399573,0.204461,...,-0.070994,-0.526048,0.069511,0.118016,-0.487334,0.052818,-0.172470,-0.160205,0.018792,-0.157210
4,-0.072722,-0.040614,0.162553,0.280708,-0.041595,0.039035,-0.042355,0.108250,-0.399030,0.205839,...,-0.070587,-0.526335,0.069912,0.117160,-0.487788,0.053419,-0.173038,-0.159827,0.017888,-0.156648


In [34]:
with open(f"dcmf_emb_CellLine_sample_{sample_no}.dat", "w") as file:
    file.write("\n")
    for idx, row in emb_df.iterrows():
        emb = row[:].astype(np.float32)
        emb_str = ' '.join(emb.astype(str))
        file.write(f'{idx}\t{emb_str}\n')
    